This code was originally written by Michael Tetley (https://github.com/headmetal) and has been slightly altered by me.

It takes an excel spreadsheet of palaeomagnetic data and turns it into a GPlates compatible gpml file that can be loaded in and tested against a reconstruction. In order to create the gpml file, at the minimum the spreadsheet must have:

    Craton name
    Name of unit/pole
    Old age
    Young age
    Saple site lat (Slat or Glat)
    Sample site lon (Slon or Glon)
    Pole lat (Plat)
    Pole lon (Plon)
    A95

It should be reasonably adaptable to other times/palaeomag datasets.

By default for VGPs GPlates uses a nominal age with a 5 Ma window for visualisation of VGPs. This script does not use a nominal age, instead it creates the features with the lower and upper ages as defined by geological data.  This gives a better appreciation of the temporal uncertainty and range of the pole, but practically it means that for visualisation in GPlates you must:

    Open the 'Layer Window' (command/control L)
    Expand the palaeomag gpml layer
    Click on 'Set VGP Visibility...'
    Make sure that 'Show all VGPs at all Times' is selected
   
This will ensure that VGPs are visible based on their begin/end time.


Referneces 

Merdith, A.S., Collins, A.S., Williams, S.E., Pisarevsky, S., Foden, J.D., Archibald, D.B., Blades, M.L., Alessio, B.L., Armistead, S., Plavsa, D., Clark, C., and Müller, R.D. 2017. A full-plate global reconstruction of the Neoproterozoic. Gondwana Research, 50, pp.84-134.

Mike's upcoming palaeomag paper

In [1]:
import numpy as np
import pygplates
import pandas as pd
import os

In [2]:
#setbasedir for loading reconstruction files
basedir = '/Users/Andrew/Documents/PhD/Scripts/Python_Scripts/pyGPlates_examples/General_plate_reconstruction/Sample_data/'

#load palaeomag data
Mer17_pmag_data = pd.read_excel('%sNeoprot-poles.xlsx' % basedir)

#load polygons
Mer17_polygons = pygplates.FeatureCollection('%sMer17_1000-520Ma_shapes.gpml' % basedir)


In [3]:
Mer17_pmag_data[:5]

,colourkey,Key,Craton,Rockunit,OldAge,YoungAge,Slat,Slong,Plat,Plong,A95,Reference,Q-factor,Declination,Inclination
0,Laurentia,L1,Laurentia,Gunbarrel Intrusions combined,780,776,45.00,-110.0,14.6,127.0,3.2,"Harlan et al., 2008",1110111 (6),301.500000,-17.100000
1,Laurentia,L2,Laurentia,Uinta Formation,800,750,41.00,-110.0,0.8,161.3,4.7,"Weil et al., 2006",0110111 (5),190.222222,-4.966667
2,Laurentia,L3,Laurentia,Kwagunt Formation,748,736,36.15,-112.0,18.2,166.0,7.0,"Weil et al., 2004",1111111 (7),280.800000,30.600000
3,Laurentia,L4,Laurentia,Franklin Dykes,727,712,75.00,-82.0,8.4,163.8,2.8,"Denyszyn et al., 2009",1111111 (7),117.425000,68.333000
4,Laurentia,L5,Laurentia,Long Range Dykes,617,613,53.50,-57.5,-19.0,175.3,17.4,"Murthy et al., 1992; Hodych et al., 2004;Age: ...",1011111 (6),110.300000,56.800000


In [4]:
# Iterate through palaeomagnetic dataset and build lists of unique terranes and continental blocks
craton_list = []
block_list = []

for i, craton in enumerate(Mer17_pmag_data.Craton.values):
    #print i, craton
    if craton not in craton_list:
        
        craton_list.append(Mer17_pmag_data.Craton[i])

In [5]:
# Iterate through palaeomagnetic pole data set and build Pandas DataFrame of required data
pole_list = []

for craton in craton_list:
    
    for k, item in enumerate(Mer17_pmag_data.Craton.values):
        
        if item == craton:
            #print item
            
            #Create dictionary for DataFrame
            #These columns headings match the data file provided here, they will need to be changed for other
            #pmag tables
            pole_list.append({'Craton': Mer17_pmag_data.Craton[k], 'Name': Mer17_pmag_data.Rockunit[k], 
                              'PLat': Mer17_pmag_data.Plat[k], 'PLon': Mer17_pmag_data.Plong[k], 
                              'SLat': Mer17_pmag_data.Slat[k],'SLon': Mer17_pmag_data.Slong[k],
                              'AgeLower': Mer17_pmag_data.YoungAge[k], 'AgeUpper': Mer17_pmag_data.OldAge[k],
                              'A95': Mer17_pmag_data.A95[k], 'Reference': Mer17_pmag_data.Reference[k]})
            
#Create and display Pandas DataFrame   
raw_polesDF = pd.DataFrame(pole_list)

In [6]:
#we use a point in poylgon test to get the appropriate plateID for each pole
plateIDs = []
plates = []

count = 0
polyCount = 0

for i in xrange(0, len(raw_polesDF)):
        
    count = count + 1
   
    if pd.isnull(raw_polesDF.SLat[i]) == False:
        inputlatLon = pygplates.LatLonPoint(raw_polesDF.SLat[i], raw_polesDF.SLon[i])
        latLonPoint = pygplates.convert_lat_lon_point_to_point_on_sphere(inputlatLon)

    for feature in Mer17_polygons:
        
         for geometry in feature.get_geometries():
            polygon = pygplates.PolygonOnSphere(geometry)
            isPoly = polygon.is_point_in_polygon(latLonPoint)

            if isPoly == True:
                plateIDs.append(feature.get_reconstruction_plate_id())
                plates.append(feature.get_name())               
                polyCount = polyCount + 1

In [7]:
#Create new GPlates Feature Collection
vpgFeatureCollection = pygplates.FeatureCollection()

#loop through our dataframe and get the right information for our VGP file
#NB in GPlates you will have to change visibility to 'Set at all Times'
#this is because we set the pole visibility to it's begin and end time.  GPlates default is to have a mean age and
#a ±5 Ma window each side of it
for j in xrange(0, len(raw_polesDF)):
    print j, str(raw_polesDF.Name[j])
    vgpFeature = pygplates.Feature.create_reconstructable_feature(
                 pygplates.FeatureType.gpml_virtual_geomagnetic_pole,
                 pygplates.PointOnSphere(raw_polesDF.PLat[j],
                                         raw_polesDF.PLon[j]),
                 name = str(raw_polesDF.Name[j]),
                 reconstruction_plate_id = plateIDs[j],
                 valid_time=(np.float(raw_polesDF.AgeUpper[j]), 
                             np.float(raw_polesDF.AgeLower[j])),
                 other_properties = [(pygplates.PropertyName.create_gpml('poleA95'), 
                                      pygplates.XsDouble(raw_polesDF.A95[j])),
                                     (pygplates.PropertyName.create_gpml('averageSampleSitePosition'),
                                      pygplates.GmlPoint(pygplates.PointOnSphere(raw_polesDF.SLat[j],
                                                                                 raw_polesDF.SLon[j])))]
                 )
 
    #Add newly created feature to existing Feature Collection
    vpgFeatureCollection.add(vgpFeature)

0 Gunbarrel Intrusions combined
1 Uinta Formation
2 Kwagunt Formation
3 Franklin Dykes
4 Long Range Dykes
5 Skinner Cove Formation
6 Southern Sweden Dykes
7 Branton-Algo Anortthosite
8 Rogaland Igneous Complex
9 Hunnedalen Dykes
10 Egersund Dykes
11 Kurgashlya Formation
12 Bakeevo Formation
13 Winter coast sediments
14 Zolotitsa sediments I, Russia
15 Verkhotina sediments
16 Zolotitsa sediments II
17 Zigan Formation
18 Ust-Kirba Formation
19 Kitoi Dykes
20 Kesyussa Formation
21 Browne Formation
22 Hussar Formation
23 Mundine Dykes
24 Johnny's Creek Member
25 Angepena Formation
26 Yaltipena Formation
27 Elatina Formation, MEAN
28 Nuccaleena Formation
29 Brachina Formation
30 Bunyeroo Formation
31 Wonoka Formation
32 Huaibei Sills 890 Ma
33 Yanbian Dykes A
34 Xiaofeng Dykes
35 Yanbian Dykes B
36 Liantuo Formation
37 Nantuo Formation
38 Doushantuo Formation
39 Luakela Volcanics A
40 Mbozi Complex
41 Bahia Dykes (N+R)
42 Sugetbrak Formation
43 Zhamoketi Andesite
44 Lower Sugetbrak Formatio

In [8]:
# Generate GPML output file
gpmlOutputFile = 'Mer17_Neoproterozoic_pmag_poles.gpml'

# Check for existing output directory and create it if not found
if not os.path.exists('gpml_output'):
    os.makedirs('gpml_output')

#save file
outputFeatureCollection = pygplates.FeatureCollectionFileFormatRegistry()
outputFeatureCollection.write(vpgFeatureCollection, 'gpml_output/' + str(gpmlOutputFile))